In [3]:
import os 
current_directory = os.getcwd()
data_directory = os.path.join(current_directory,'..', "GeneratedData")
verify_directory(data_directory)
points_directory = os.path.join(data_directory, "points")
verify_directory(points_directory)
mu_sigma_directory = os.path.join(data_directory, "mu_sigma")
verify_directory(mu_sigma_directory)
data_space_directory = os.path.join(data_directory, "data_space")

In [2]:
def verify_directory(path: str):
    """
    Verify if a directory at the specified path exists and create it if not.

    Parameters:
        path (str): The path to the directory.

    Returns:
        None
    """
    if not os.path.exists(path):
        os.mkdir(path)

In [4]:
import itertools
mu = [0,5,10]
sigma = [1,3,5]
q = [1,1.5,2]
parameters = {"num_points": 100,"dimension": 50}
combinations = list(itertools.product(mu, sigma, q))

# Get parameter keys
parameter_keys = list(parameters.keys())

In [219]:
from SVMbasic import *
import pandas as pd

# space_parameters represent ((mu1,sigma1,q1),(mu2,sigma2,q2))
ts = [0.1,0.5,0.6, 0.7, 0.8, 0.9, 1]
qs = [1, 1.2, 1.5, 1.7, 2, 2.5, 2.8]

space_ix=itertools.combinations(combinations, r=2)
kernel_ix=[('rbf',''),('lineal',''),('poli','')]+[('fisher','t{:.1f}_q{:.1f}'.format(t,q)) for t,q in itertools.product(ts,qs)]
svm_results=pd.DataFrame(columns=['accuracy'],
                         index=pd.MultiIndex.from_tuples([tuple(itertools.chain(*row)) for row in itertools.product(space_ix,kernel_ix)],
                                                         names=['(mu0,sigma0,q0)','(mu1,sigma1,q1)','SVM kernel','kernel parameters']))
svm_results

accuracy
(mu0,sigma0,q0) (mu1,sigma1,q1) SVM kernel kernel parameters         
(0, 1, 1)       (0, 1, 1.5)     rbf                               NaN
                                lineal                            NaN
                                poli                              NaN
                                fisher     t0.1_q1.0              NaN
                                           t0.1_q1.2              NaN
...                                                               ...
(10, 5, 1.5)    (10, 5, 2)      fisher     t1.0_q1.5              NaN
                                           t1.0_q1.7              NaN
                                           t1.0_q2.0              NaN
                                           t1.0_q2.5              NaN
                                           t1.0_q2.8              NaN

[18252 rows x 1 columns]

In [223]:
from SVMfisher import *

for theta_0, theta_1 in svm_results.index.droplevel(['SVM kernel','kernel parameters']).drop_duplicates():
    # Load data
    df_ij=pd.read_csv(os.path.join(data_space_directory,'mu%s_sigma%s_q%s_mu%s_sigma%s_q%s.csv'%(*theta_0,*theta_1)))
    x=df_ij[['mu','sigma']].to_numpy()
    y=df_ij[['label']].to_numpy().squeeze()

    # Train SVMs with known kernels
    svms = train_svms(x,y)
    kernels_accuracy=evaluate_kernels(svms, x, y)
    svm_results.loc[(theta_0,theta_1,['rbf','lineal','poli']),:]=list(kernels_accuracy)

svm_results.loc[(slice(None),slice(None),['rbf','lineal','poli']),:]

accuracy
(mu0,sigma0,q0) (mu1,sigma1,q1) SVM kernel kernel parameters         
(0, 1, 1)       (0, 1, 1.5)     rbf                              0.96
                                lineal                           0.96
                                poli                            0.975
                                fisher     t0.1_q1.0            0.475
                                           t0.1_q1.2            0.475
...                                                               ...
(10, 5, 1.5)    (10, 5, 2)      fisher     t1.0_q1.5              NaN
                                           t1.0_q1.7              NaN
                                           t1.0_q2.0              NaN
                                           t1.0_q2.5              NaN
                                           t1.0_q2.8              NaN

[18252 rows x 1 columns]

In [226]:
svm_results.loc[(slice(None),slice(None),['rbf','lineal','poli']),:].to_csv(os.path.join(data_directory,'svm_results_ex.csv'))

In [228]:
from SVMfisher import *

for theta_0, theta_1 in svm_results.index.droplevel(['SVM kernel','kernel parameters']).drop_duplicates():
    # Load data
    df_ij=pd.read_csv(os.path.join(data_space_directory,'mu%s_sigma%s_q%s_mu%s_sigma%s_q%s.csv'%(*theta_0,*theta_1)))
    x=df_ij[['mu','sigma']].to_numpy()
    y=df_ij[['label']].to_numpy().squeeze()

    #f_acc=run_all_configurations_from_space(x, y, ts, qs)
    svm_results.loc[(theta_0, theta_1,'fisher', 't'+f_acc['t'].astype(str)+'_q'+f_acc['q'].astype(str)),:]=f_acc['fisher'].to_numpy()
    

In [231]:
svm_results.loc[(slice(None), slice(None), 'fisher', slice(None)),:]

accuracy
(mu0,sigma0,q0) (mu1,sigma1,q1) SVM kernel kernel parameters         
(0, 1, 1)       (0, 1, 1.5)     fisher     t0.1_q1.0            0.475
                                           t0.1_q1.2            0.475
                                           t0.1_q1.5              NaN
                                           t0.1_q1.7              NaN
                                           t0.1_q2.0              NaN
...                                                               ...
(10, 5, 1.5)    (10, 5, 2)      fisher     t1.0_q1.5              NaN
                                           t1.0_q1.7              NaN
                                           t1.0_q2.0              NaN
                                           t1.0_q2.5              NaN
                                           t1.0_q2.8              NaN

[17199 rows x 1 columns]